# Pre-process French C2C Data - mk2

#### 16 May 2022

This notebook annotates the preprocessing of a dataset to ensure quality data while attempting to minimize injected technical bias. No data has been imputed. Two versions will result: one with explicit one-hot-encoding for categorical variables and one without.

## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
pd.set_option('display.max_rows',500)

### Versions

In [1]:
print("*** Versions ***")
print("numpy:\t\t", np.__version__)
print("pandas:\t\t", pd.__version__)

*** Versions ***


NameError: name 'np' is not defined

## Import Data

In [4]:
data = pd.read_csv("frenchC2C.csv")

This dataset is from a C2C (consumer-to-consumer) e-commerce store. It was scraped from the Vestiaire Collective website, an international second-hand (or "pre-loved") fashion app which begian in Europe in 2009. It has since expanded to over nine million registered users around the world.

The data consists of 98913 samples with 24 features. The target variable, *productsBought*, is an integer with range from \[0,$\infty$\). It has no missing values.

## Custom Functions

#### Save to persistent storage

In [ ]:
def custom_save(name, data, kind=1):
    '''
    name : string
        designated filename
    data : data or pytorch model
        the data to save
    kind : int
        sentinel value - 1 if pytorch model, 0 otherwise
    
    custom_save stores the data passed into the function into a file with the provided name
    '''
    
    if kind == 1:
        ex = ".pth"
    else:
        ex = ".parquet"
    
    sentinel = True
    i = 1

    while sentinel:
        dirlist = os.listdir()

        if name not in dirlist:
            if kind == 1:
                torch.save(data, name)
            else:
                data.to_parquet(name)
            print(f"{name} has been saved.")                
            sentinel = False
        if name in dirlist:
            print(f"{name} already exists.", end=" ")
            temp, ext = name.split(ex)
            if "_v" in temp:
                temp, _ = temp.split("_v")
            name = f"{temp}_v{i}{ex}"
            i = i + 1
            print(f"Changing file name to: {name}")

## Data Cleaning

In [5]:
data.shape

(98913, 24)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98913 entries, 0 to 98912
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   identifierHash       98913 non-null  int64  
 1   type                 98913 non-null  object 
 2   country              98913 non-null  object 
 3   language             98913 non-null  object 
 4   socialNbFollowers    98913 non-null  int64  
 5   socialNbFollows      98913 non-null  int64  
 6   socialProductsLiked  98913 non-null  int64  
 7   productsListed       98913 non-null  int64  
 8   productsSold         98913 non-null  int64  
 9   productsPassRate     98913 non-null  float64
 10  productsWished       98913 non-null  int64  
 11  productsBought       98913 non-null  int64  
 12  gender               98913 non-null  object 
 13  civilityGenderId     98913 non-null  int64  
 14  civilityTitle        98913 non-null  object 
 15  hasAnyApp            98913 non-null 

In [7]:
data.describe()

,identifierHash,socialNbFollowers,socialNbFollows,socialProductsLiked,productsListed,productsSold,productsPassRate,productsWished,productsBought,civilityGenderId,daysSinceLastLogin,seniority,seniorityAsMonths,seniorityAsYears
count,9.891300e+04,98913.000000,98913.000000,98913.000000,98913.000000,98913.000000,98913.000000,98913.000000,98913.000000,98913.000000,98913.000000,98913.000000,98913.000000,98913.000000
mean,-6.692039e+15,3.432269,8.425677,4.420743,0.093304,0.121592,0.812303,1.562595,0.171929,1.773993,581.291236,3063.771870,102.125583,8.510424
std,5.330807e+18,3.882383,52.839572,181.030569,2.050144,2.126895,8.500205,25.192793,2.332266,0.428679,208.855888,168.298621,5.609735,0.467863
min,-9.223101e+18,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,11.000000,2852.000000,95.070000,7.920000
25%,-4.622895e+18,3.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,572.000000,2857.000000,95.230000,7.940000
50%,-1.337989e+15,3.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,694.000000,3196.000000,106.530000,8.880000
75%,4.616388e+18,3.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,702.000000,3201.000000,106.700000,8.890000
max,9.223331e+18,744.000000,13764.000000,51671.000000,244.000000,174.000000,100.000000,2635.000000,405.000000,3.000000,709.000000,3205.000000,106.830000,8.900000


#### Inspect values

In [ ]:
# if the type of column is "object", then display what its values are
for c in data.columns:
    if data[c].dtype == object:
        print(f"{c}: {data[c].dtype}")
        print(data[c].unique())
        print()

#### identifierHash, type

In [8]:
data.nunique()

identifierHash         98913
type                       1
country                  200
language                   5
socialNbFollowers         90
socialNbFollows           85
socialProductsLiked      420
productsListed            65
productsSold              75
productsPassRate          72
productsWished           279
productsBought            70
gender                     2
civilityGenderId           3
civilityTitle              3
hasAnyApp                  2
hasAndroidApp              2
hasIosApp                  2
hasProfilePicture          2
daysSinceLastLogin       699
seniority                 19
seniorityAsMonths         19
seniorityAsYears           6
countryCode              199
dtype: int64

##### Actions
1) *type* is singular, providing no information gain. conversely, *identifierHash* values are unique for each sample.
<ul>- drop <i>type</i>, replace <i>identifierHash</i> with a simpler, serialized row count as id number</ul>

In [9]:
data.drop(['identifierHash', 'type'], axis=1, inplace=True)

#### Gender, civilityTitle, civilityIGenderId

In [10]:
data.civilityGenderId.unique()

array([1, 2, 3])

In [11]:
data.civilityTitle.unique()

array(['mr', 'mrs', 'miss'], dtype=object)

In [12]:
data.gender.unique()

array(['M', 'F'], dtype=object)

In [13]:
data[['gender','civilityTitle','civilityGenderId']].head(10)

,gender,civilityTitle,civilityGenderId
0,M,mr,1
1,F,mrs,2
2,F,mrs,2
3,F,mrs,2
4,F,mrs,2
5,F,mrs,2
6,F,miss,3
7,F,mrs,2
8,F,mrs,2
9,F,miss,3


##### Actions
2) *civilityGenderId* and *civlityTitle* represent the same data. *civilityGenderId* captures female marital status. However, the usage of "miss"/"mademoiselle" vs "mrs"/"madame" is contingent on location, age, and culture of user.
<ul>- drop <i>civilityTitle</i> </ul>
<ul>- rename <i>civilityGenderId</i> name to <i>civGen</i>

In [14]:
data.drop('civilityTitle', axis=1, inplace=True)

In [15]:
data.rename(columns={'civilityGenderId':'civGen'}, inplace=True)

#### Country

In [16]:
data.language.unique()

array(['en', 'fr', 'de', 'it', 'es'], dtype=object)

In [17]:
data.country.unique()

array(['Royaume-Uni', 'Monaco', 'France', 'Etats-Unis', 'Allemagne',
       'Suède', 'Italie', 'Espagne', 'Croatie', 'Lettonie', 'Taiwan',
       'Bulgarie', 'Chypre', 'Luxembourg', 'Roumanie', 'Pays-Bas',
       'Slovaquie', 'Hong Kong', 'Autriche', 'Belgique', 'Suisse',
       'Australie', 'Pologne', 'Grèce', 'Portugal', 'Russie', 'Finlande',
       'Slovénie', 'Danemark', 'Serbie', 'Irlande', 'Canada', 'Singapour',
       'Ukraine', 'Lituanie', 'Chine', 'Japon', 'Estonie', 'Islande',
       'Kazakhstan', 'Bahamas', 'Îles Canaries', 'Guam', 'Malaisie',
       'Brunei Darussalam', 'Viet Nam', 'Hongrie', 'Émirats arabes unis',
       'Bahreïn', 'Macau', 'Honduras', 'Arabie Saoudite', 'Brésil',
       'Norvège', 'Liban', 'Cambodge', 'Inde', 'Mexique', 'Corée du Sud',
       'Kiribati', 'République tchèque', 'Nouvelle Zélande', 'Egypte',
       'Puerto Rico', 'Albanie', 'Guadeloupe', 'Laos', 'Turquie',
       'Paraguay', 'Maroc', 'Nigeria', 'Colombie', 'Myanmar',
       'Afghanistan', 'G

In [18]:
data.countryCode.unique()

array(['gb', 'mc', 'fr', 'us', 'de', 'se', 'it', 'es', 'hr', 'lv', 'tw',
       'bg', 'cy', 'lu', 'ro', 'nl', 'sk', 'hk', 'at', 'be', 'ch', 'au',
       'pl', 'gr', 'pt', 'ru', 'fi', 'si', 'dk', 'rs', 'ie', 'ca', 'sg',
       'ua', 'lt', 'cn', 'jp', 'ee', 'is', 'kz', 'bs', 'ic', 'gu', 'my',
       'bn', 'vn', 'hu', 'ae', 'bh', 'mo', 'hn', 'sa', 'br', 'no', 'lb',
       'kh', 'in', 'mx', 'kr', 'ki', 'cz', 'nz', 'eg', 'pr', 'al', 'gp',
       'la', 'tr', 'py', 'ma', 'ng', 'co', 'mm', 'af', 'ga', 'er', 'ph',
       'am', 'mn', 'tn', 'ag', 'dz', 'cl', 'qa', 're', 'vg', 'je', 'id',
       'th', 'md', 'na', 'ci', 'gn', 'ge', 'jo', 'za', 'bj', 'kw', 'mq',
       'ao', 'bm', 'zw', 'az', 'il', 'ec', 'ar', 'mt', 'bf', 'as', 've',
       'cr', 'ba', 'vu', 'pe', 'do', 'gg', 'gy', 'bb', 'pk', 'cg', 'mg',
       'dj', 'sn', 'pa', 'tt', 'rw', 'sv', 'ml', 'ne', 'cm', 'nc', 'ky',
       'lk', 'bo', 'pf', 'ug', 'tg', 'sj', 'tj', 'cu', 'gt', 'ai', 'tc',
       'ke', 'jm', 'by', 'om', 'tz', 'bz', 'cf', 'b

##### Note
There are 200 countries and 199 country codes. Search for possible error. Most likely, a country is either mispelled or has an alternate spelling. Compare country codes to country names to find the outlier.

In [19]:
# group dataset by country and country code for comparison

df= data.groupby(['country','countryCode'])['countryCode'].count()
df

country                                 countryCode
Afghanistan                             af                10
Afrique du Sud                          za                83
Albanie                                 al                37
Algérie                                 dz                76
Allemagne                               de              6567
Andorre                                 ad                 9
Angola                                  ao                 6
Anguilla                                ai                 2
Antarctique                             aq                 1
Antigua et Barbuda                      ag                 3
Antilles néerlandaises                  an                 1
Arabie Saoudite                         sa               143
Argentine                               ar                39
Arménie                                 am                16
Aruba                                   aw                 3
Australie                        

In [20]:
# search for duplicate country code and display the associated country names

for i in range(0, len(df)-1):
    if (df.index[i][1] == df.index[i+1][1]):
        print(f"Mismatch :{df[i]}\t{df[i+1]}")
        print(f"\t{df.index[i]}")
        print(f"\t{df.index[i+1]}")
        

Mismatch :1	14
	('Guyana', 'gy')
	('Guyane', 'gy')


##### Actions
3) Guyana and Guyane are the same country. As such, having both *country* and *countryCode* is redundant; they express the same feature. *countryCode* is kept for human readability
<ul>- drop <i>country</i></ul>
4) clean up data space
<ul>- delete intermediary dataframe <i>df</i></ul>

In [21]:
del df
data.drop('country', axis=1, inplace=True)

#### seniority, seniorityAsMonths, seniorityAsYears

In [22]:
data.nunique()

language                 5
socialNbFollowers       90
socialNbFollows         85
socialProductsLiked    420
productsListed          65
productsSold            75
productsPassRate        72
productsWished         279
productsBought          70
gender                   2
civGen                   3
hasAnyApp                2
hasAndroidApp            2
hasIosApp                2
hasProfilePicture        2
daysSinceLastLogin     699
seniority               19
seniorityAsMonths       19
seniorityAsYears         6
countryCode            199
dtype: int64

In [23]:
s = data.seniority.unique()
s

array([3196, 3204, 3203, 3198, 2854, 2857, 3195, 2856, 2855, 2859, 2853,
       2858, 3205, 3202, 3197, 3200, 3201, 3199, 2852])

In [24]:
m = data.seniorityAsMonths.unique()
m

array([106.53, 106.8 , 106.77, 106.6 ,  95.13,  95.23, 106.5 ,  95.2 ,
        95.17,  95.3 ,  95.1 ,  95.27, 106.83, 106.73, 106.57, 106.67,
       106.7 , 106.63,  95.07])

In [25]:
y = data.seniorityAsYears.unique()
y

array([8.88, 8.9 , 7.93, 7.94, 7.92, 8.89])

In [26]:
np.unique(np.round(data.seniorityAsMonths.unique()/12,2))

array([7.92, 7.93, 7.94, 8.88, 8.89, 8.9 ])

In [27]:
# display the seniority and seniorityAsMonths side by side
list(zip(s,m))

[(3196, 106.53),
 (3204, 106.8),
 (3203, 106.77),
 (3198, 106.6),
 (2854, 95.13),
 (2857, 95.23),
 (3195, 106.5),
 (2856, 95.2),
 (2855, 95.17),
 (2859, 95.3),
 (2853, 95.1),
 (2858, 95.27),
 (3205, 106.83),
 (3202, 106.73),
 (3197, 106.57),
 (3200, 106.67),
 (3201, 106.7),
 (3199, 106.63),
 (2852, 95.07)]

In [28]:
np.round(s/m)

array([30., 30., 30., 30., 30., 30., 30., 30., 30., 30., 30., 30., 30.,
       30., 30., 30., 30., 30., 30.])

##### Actions
5) *seniorityAsYears* is derived/derivable directly from *seniorityAsMonths*. Remove *seniorityAsYears* for redundancy.

6) *seniority* = round(*seniorityAsMonths* * 30), displaying seniority an **int** value. Remove *seniority* for redundancy
<ul> -remove <i>seniorityAsYears</i> and <i>seniority</i></ul>
7) clean data space
<ul>- delete s, m, y variables </ul>

In [29]:
data.drop(['seniority','seniorityAsYears'], axis=1, inplace=True)

In [30]:
del s, m, y

#### productsBought

In [31]:
data.productsBought.nunique()

70

In [32]:
data.productsBought.unique()

array([  1,   0,   3, 105,   2,  36,  32,  14, 115,   6,   8,  69,  12,
         9,   5,  80,  13,  20,   7,   4,  54,  11,  48,  31,  28,  16,
       279,  30,  37,  10, 174,  33,  23,  65,  40,  25,  27,  41,  53,
        19,  22,  18,  85,  15,  24,  58,  17,  38,  21,  74,  47,  57,
        67, 405,  77,  70,  81,  66,  52,  50,  35,  73,  87,  44,  39,
        93,  51,  26,  34,  55])

*productsBought* is the target variable. We aim to predict whether a purchase will occur, not how many purchases. As such, we will engineer a feature, *target*, to capture the binary values of "not purchased"/"purchased" associated with *productsBought* in the boolean set \{0,1\}

In [33]:
data[data.productsBought > 1].productsBought

2          3
6        105
9          2
12        36
16        32
        ... 
98199      7
98213      2
98469      4
98794      2
98898      5
Name: productsBought, Length: 2122, dtype: int64

##### Action
8) create *target*. It will be a boolean mask for purchases. If there is more than 0 purchases, it will be marked as 1, else as 0

In [34]:
data['target'] = data.productsBought.apply(lambda x: x > 0).astype(int)

In [35]:
data[['productsBought','target']].head(20)

,productsBought,target
0,1,1
1,0,0
2,3,1
3,0,0
4,0,0
5,0,0
6,105,1
7,0,0
8,0,0
9,2,1


### Save one-hot-encoded dataset

In [37]:
data.columns.to_list()

['language',
 'socialNbFollowers',
 'socialNbFollows',
 'socialProductsLiked',
 'productsListed',
 'productsSold',
 'productsPassRate',
 'productsWished',
 'productsBought',
 'gender',
 'civGen',
 'hasAnyApp',
 'hasAndroidApp',
 'hasIosApp',
 'hasProfilePicture',
 'daysSinceLastLogin',
 'seniorityAsMonths',
 'countryCode',
 'target']

In [38]:
df = pd.get_dummies(data, columns=['civGen','gender','countryCode','language'])

In [39]:
df.columns.to_list()

['socialNbFollowers',
 'socialNbFollows',
 'socialProductsLiked',
 'productsListed',
 'productsSold',
 'productsPassRate',
 'productsWished',
 'productsBought',
 'hasAnyApp',
 'hasAndroidApp',
 'hasIosApp',
 'hasProfilePicture',
 'daysSinceLastLogin',
 'seniorityAsMonths',
 'target',
 'civGen_1',
 'civGen_2',
 'civGen_3',
 'gender_F',
 'gender_M',
 'countryCode_ad',
 'countryCode_ae',
 'countryCode_af',
 'countryCode_ag',
 'countryCode_ai',
 'countryCode_al',
 'countryCode_am',
 'countryCode_an',
 'countryCode_ao',
 'countryCode_aq',
 'countryCode_ar',
 'countryCode_as',
 'countryCode_at',
 'countryCode_au',
 'countryCode_aw',
 'countryCode_az',
 'countryCode_ba',
 'countryCode_bb',
 'countryCode_bd',
 'countryCode_be',
 'countryCode_bf',
 'countryCode_bg',
 'countryCode_bh',
 'countryCode_bj',
 'countryCode_bl',
 'countryCode_bm',
 'countryCode_bn',
 'countryCode_bo',
 'countryCode_br',
 'countryCode_bs',
 'countryCode_bv',
 'countryCode_bw',
 'countryCode_by',
 'countryCode_bz',
 'co

In [ ]:
name = "french.parquet"
custom_save(name, df, 0)